Element resolution from an arbitrary spectra is likely going to be a multivariate fitting so this notebook explores available optimization libraries in python. 
Advancing LIBS to calibration free estimation may involve allowing for variation in plasma conditions by element and time. For example, if certain ions are emitting at differnt points then the plasma temp/density can vary across the lines observed. In fitting the observed data we may be able to estimate these parameters from relative line strengths observed. Alternatively we could allow these parameters to vary to determine optimal fit. The implied sample elemental composition will be better fit.

Breaking each element down into ionization states and the associated energies/probabilities as f(Temp,dens) each could be fit separately and normalized to a reference Temp/Dens upon which full elemental composition could be estimated.

SciPy is a good starting point for optimization routines, though there are additional python libraries available to review.
https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [2]:
import numpy as np
from scipy.optimize import minimize

In [3]:
#objective function must have signature f(x, *args) where x is numpy array
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [4]:
#nelder-mead is a simplex algorithm and does not use gradient (Powell also needs only function calls)
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [5]:
print(res.x)

[1. 1. 1. 1. 1.]


In [6]:
#Faster convergence from methods that include a gradient, Can be passed or estimated from first diffs
#Broyden-Fletcher-Goldfarb-Shanno algorithm (method='BFGS')
res = minimize(rosen, x0, method='BFGS', options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 25
         Function evaluations: 180
         Gradient evaluations: 30


In [7]:
#include the gradient explicitly, best in the objective def which now returns a tuple
def rosen_and_der(x):
    objective = sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return objective, der

In [8]:
#note if sep fxn for jacobian, specify that as jac arg
res = minimize(rosen_and_der, x0, method='BFGS', jac=True, options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 25
         Function evaluations: 30
         Gradient evaluations: 30


In [11]:
#standalond jacobian for next method call
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

In [9]:
#Newton-conjugate takes second derivative into account via Hessian matrix
#Basically a two-term Taylor expansion
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

In [12]:
#note, another form allows for hessp= which is function giving product of Hessian with arb vector p
res = minimize(rosen, x0, method='Newton-CG',
                jac=rosen_der, hess=rosen_hess,
                options={'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 24


In [ ]:
#Trust-Region Newton-Conjugate-Gradient Algorithm
